In [ ]:

plate1 = {'dish': 'Hummus', 'price': 800, 'quantity': 5}


In [ ]:

plate2 = {'dish': 'Avocado', 'price': 900, 'quantity': 2}
plates = [ plate1, plate2 ]


In [ ]:

len(plates), plates[0]['dish']


In [ ]:

plates == [ {'dish': 'Hummus', 'price': 800, 'quantity': 5}, {'dish': 'Avocado', 'price': 900, 'quantity': 2} ]


In [ ]:

from json import *

s  = dumps(plates)
plates, s


In [ ]:

plates1 = loads(s)
plates == plates1


In [ ]:

from pandas import *

plates_df = DataFrame(plates)
plates_df


In [ ]:

plates1 =  plates_df.to_dict('records')
plates == plates1


In [ ]:

menu_df = read_csv('menu.csv')
menu_df


In [ ]:

menu_df.to_csv('menu1.csv') 
menu_df.to_csv('menu2.csv', index = False) 


In [ ]:

# prompt for  database account password
from getpass import getpass

user =  'admin'
passwd = getpass()
server =  'localhost'
dialect, driver = 'mysql', 'pymysql'
url = f'{dialect}+{driver}://{user}:{passwd}@{server}'
url


In [ ]:

from sqlalchemy import create_engine

engine = create_engine(url)


In [ ]:

%load_ext sql


In [ ]:

%sql engine


In [ ]:

%sql show databases;


In [ ]:

%%sql 

drop database if exists rawa;
create database rawa;
show databases;


In [ ]:

%sql --persist --no-index rawa.menu_df 


In [ ]:

%%sql 

rename table rawa.menu_df to rawa.Menu;
show tables from rawa;


In [ ]:

%%sql

use rawa;
select * from Menu;


In [ ]:

output = %sql select * from Menu;
menu_df = DataFrame(output)
menu_df


In [ ]:

menu_df = read_csv('menu.csv')
records_df = read_csv('records.csv')
menu = menu_df.to_dict('records')
records = records_df.to_dict('records')

def tally_plate(item, record):
	dish = item['dish']
	price, quantity = item['price'], record[dish]
	del(record[dish])
	plate = { 'dish': dish, 'price': price, 'quantity': quantity }
	return plate
	
def extract_orders(record):
	keys = [ 'orderId', 'created', 'customerId', 'tip' ]
	order = { key:record[key] for key in keys }
	plates = [ tally_plate(item, record) for item in menu if item['dish'] in record and record[item['dish']] ]
	subtotal = int(sum([ plate['price']*plate['quantity'] for plate in plates ]))
	order['subtotal'] = subtotal
	tip = order['tip']
	tax = .0735*(tip + subtotal)
	total = subtotal + tip + tax
	order['tax'] = int(tax)
	order['total'] = int(total)
	order['plates'] = dumps(plates)
	return order

orders  =  [ extract_orders(record) for record in records ]


In [ ]:

orders_df = DataFrame(orders)

cols = [ 'orderId', 'created', 'customerId', 'plates' ]
ordersin_df  = orders_df[cols]

cols = [ 'orderId', 'tip', 'subtotal', 'tax', 'total' ]
ordersout_df = orders_df[cols]



In [ ]:

user = 'admin'
passwd = getpass()
server = 'localhost'
url = f'mysql+pymysql://{user}:{passwd}@{server}'

engine = create_engine(url)

%load_ext sql
%sql engine


In [ ]:

%sql drop database if exists rawa;
%sql create database rawa;

tables = {'menu_df': 'Menu', 'ordersin_df': 'OrdersIn', 'ordersout_df': 'OrdersOut' }

for key in tables:
	%sql --persist --no-index rawa.{{key}}
	value = tables[key]
	%sql rename table rawa.{{key}} to rawa.{{value}}


In [ ]:

%%sql

use rawa;
show tables;
